In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import numpy as np
import pandas as pd
import attacks
import models
import utils
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, TensorDataset
from torch import Tensor
from tqdm.notebook import trange
import plotly.graph_objects as go

utils.create_train_val()
# dataloaders, dataset_sizes   = utils.task_loader()

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 
device

  0%|          | 0/60000 [00:00<?, ?it/s]

NameError: name 'TensorDataset' is not defined

In [7]:
ds = torch.load('./train.pt')
online = ds[30000:]
online = TensorDataset( online[0], online[1],online[2] )
offline = ds[:30000]
offline = TensorDataset( offline[0], offline[1],offline[2] )

In [26]:
controler = models.load_controler()
target = models.load_target()

n_epochs = 50
bar = trange(n_epochs)
epoch_loss = {'train': 0, 'val': 0}
dft = pd.DataFrame(columns=['train_bce', 'val_bce'])

optimizer = optim.Adam(controler.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

for epoch in bar:
    
    # Each epoch has a training and validation phase
    for phase in ['train', 'val']: 
        if phase == 'train':
            controler.train()  # Set model to training mode
        else:
            controler.eval()   # Set model to evaluate mode

        running_loss = 0.0
        bar.set_description(f'Epoch {epoch} {phase}'.ljust(20))

        # Iterate over data.
        for X,b,y in dataloaders[phase]:
            X,y = X.to(device), y.to(device)
     
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                
                yp = controler( X )    
                loss = torch.nn.BCELoss()( yp, y )

                # backward + optimize only if in training phase
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
                    
                running_loss += loss.item() * X.size(0)
                epoch_loss[phase] = running_loss / dataset_sizes[phase]

        if phase == 'train':
            scheduler.step()
#             print(scheduler.get_last_lr())
            
        bar.set_postfix( train_loss=f'{epoch_loss["train"]:0.5f}', val_loss=f'{epoch_loss["val"]:0.5f}') 
        dft.loc[epoch, f'{phase}_bce'] = epoch_loss[phase]
        
        status = {
            "epoch": epoch,
            "model_state": controler.state_dict(),
            "optimizer_state": optimizer.state_dict() }
        
        torch.save(status, './offline_controler.pt')

  0%|          | 0/50 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [24]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dft.index,
    y=dft.train_bce,
    name = 'Train BCE' ))
fig.add_trace(go.Scatter(
    x=dft.index,
    y=dft.val_bce,
    name='Val BCE' ))
fig.update_yaxes(range=[0, 0.2])

fig.show()